<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo">
</center>


#### Import the required libraries we need for the lab.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np

#### Read the dataset in the csv file from the URL


In [ ]:
boston_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ST0151EN-SkillsNetwork/labs/boston_housing.csv')

## Task 1: Familiarize yourself with the dataset

Let's explore the dataset structure and basic information.

In [ ]:
# Display first few rows
print("First 5 rows of the dataset:")
print(boston_df.head())

In [ ]:
# Dataset information
print("\nDataset Information:")
print(boston_df.info())

In [ ]:
# Basic statistics
print("\nBasic Statistics:")
print(boston_df.describe())

In [ ]:
# Check for missing values
print("\nMissing Values:")
print(boston_df.isnull().sum())

## Task 2: Generate Descriptive Statistics and Visualizations

### 2.1 Boxplot for Median Value of Owner-Occupied Homes (MEDV)

In [ ]:
# Boxplot for MEDV
plt.figure(figsize=(10, 6))
plt.boxplot(boston_df['MEDV'], vert=True, patch_artist=True)
plt.ylabel('Median Value ($1000s)', fontsize=12)
plt.title('Boxplot of Median Value of Owner-Occupied Homes', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nStatistics for MEDV:")
print(f"Mean: ${boston_df['MEDV'].mean():.2f}k")
print(f"Median: ${boston_df['MEDV'].median():.2f}k")
print(f"Standard Deviation: ${boston_df['MEDV'].std():.2f}k")
print(f"Min: ${boston_df['MEDV'].min():.2f}k")
print(f"Max: ${boston_df['MEDV'].max():.2f}k")
print(f"\nFinding: The median home value is around ${boston_df['MEDV'].median():.2f}k with some outliers on the higher end.")
print("The distribution shows that most homes are valued between $17k and $25k.")

### 2.2 Bar Plot for Charles River Variable (CHAS)

In [ ]:
# Bar plot for CHAS
chas_counts = boston_df['CHAS'].value_counts()

plt.figure(figsize=(10, 6))
bars = plt.bar(['Not on Charles River (0)', 'On Charles River (1)'], chas_counts.values, 
               color=['steelblue', 'coral'], edgecolor='black', linewidth=1.5)
plt.ylabel('Number of Houses', fontsize=12)
plt.title('Distribution of Houses by Charles River Location', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nCharles River Statistics:")
print(f"Houses NOT on Charles River: {chas_counts[0]} ({chas_counts[0]/len(boston_df)*100:.1f}%)")
print(f"Houses ON Charles River: {chas_counts[1]} ({chas_counts[1]/len(boston_df)*100:.1f}%)")
print(f"\nFinding: Only a small proportion ({chas_counts[1]/len(boston_df)*100:.1f}%) of houses bound the Charles River.")
print("This makes riverside properties relatively rare in the dataset.")

### 2.3 Boxplot for MEDV vs AGE (Discretized into 3 Groups)

In [ ]:
# Discretize AGE variable into three groups
boston_df['AGE_Group'] = pd.cut(boston_df['AGE'], 
                                 bins=[0, 35, 70, 100], 
                                 labels=['35 years and younger', 'Between 35 and 70 years', '70 years and older'])

# Boxplot for MEDV vs AGE groups
plt.figure(figsize=(12, 6))
boston_df.boxplot(column='MEDV', by='AGE_Group', patch_artist=True, grid=False)
plt.xlabel('Age Group (Proportion of Units Built Before 1940)', fontsize=12)
plt.ylabel('Median Value ($1000s)', fontsize=12)
plt.title('Median Home Value by Age of Housing Units', fontsize=14, fontweight='bold')
plt.suptitle('')  # Remove the default title
plt.xticks(rotation=15, ha='right')
plt.tight_layout()
plt.show()

print("\nMedian Value by Age Group:")
for group in boston_df['AGE_Group'].cat.categories:
    group_data = boston_df[boston_df['AGE_Group'] == group]['MEDV']
    print(f"\n{group}:")
    print(f"  Mean: ${group_data.mean():.2f}k")
    print(f"  Median: ${group_data.median():.2f}k")
    print(f"  Count: {len(group_data)}")

print("\nFinding: There appears to be a relationship between housing age and median value.")
print("Newer developments tend to have different value distributions compared to older areas.")

### 2.4 Scatter Plot: NOX vs INDUS

In [ ]:
# Scatter plot for NOX vs INDUS
plt.figure(figsize=(10, 6))
plt.scatter(boston_df['INDUS'], boston_df['NOX'], alpha=0.5, edgecolors='black', linewidth=0.5)
plt.xlabel('Proportion of Non-Retail Business Acres per Town', fontsize=12)
plt.ylabel('Nitric Oxide Concentration (parts per 10 million)', fontsize=12)
plt.title('Relationship between Nitric Oxide Concentration and Industrial Areas', 
          fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate correlation
correlation = boston_df['NOX'].corr(boston_df['INDUS'])
print(f"\nCorrelation between NOX and INDUS: {correlation:.4f}")
print(f"\nFinding: There is a strong positive relationship (r = {correlation:.4f}) between")
print("industrial areas and nitric oxide concentrations. As the proportion of non-retail")
print("business acres increases, nitric oxide pollution tends to increase as well.")

### 2.5 Histogram for Pupil-Teacher Ratio

In [ ]:
# Histogram for PTRATIO
plt.figure(figsize=(10, 6))
plt.hist(boston_df['PTRATIO'], bins=20, edgecolor='black', color='skyblue', alpha=0.7)
plt.xlabel('Pupil-Teacher Ratio', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Pupil-Teacher Ratio by Town', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nPupil-Teacher Ratio Statistics:")
print(f"Mean: {boston_df['PTRATIO'].mean():.2f}")
print(f"Median: {boston_df['PTRATIO'].median():.2f}")
print(f"Standard Deviation: {boston_df['PTRATIO'].std():.2f}")
print(f"Min: {boston_df['PTRATIO'].min():.2f}")
print(f"Max: {boston_df['PTRATIO'].max():.2f}")
print(f"\nFinding: Most towns have pupil-teacher ratios between 18 and 21.")
print("The distribution shows some variation in educational resources across towns.")

## Task 3: Hypothesis Testing

### 3.1 T-Test: Is there a significant difference in median value of houses bounded by the Charles River?

In [ ]:
print("="*80)
print("T-TEST FOR INDEPENDENT SAMPLES")
print("="*80)

# Separate data by Charles River
medv_river = boston_df[boston_df['CHAS'] == 1]['MEDV']
medv_no_river = boston_df[boston_df['CHAS'] == 0]['MEDV']

print("\n1. HYPOTHESIS:")
print("   H0 (Null Hypothesis): There is no significant difference in median home values")
print("                         between houses bounded by Charles River and those not.")
print("   H1 (Alternative Hypothesis): There IS a significant difference in median home values")
print("                                between the two groups.")

print("\n2. SIGNIFICANCE LEVEL:")
print("   α = 0.05")

print("\n3. TEST STATISTICS:")
# Perform independent t-test
t_statistic, p_value = scipy.stats.ttest_ind(medv_river, medv_no_river)

print(f"   Sample size (Charles River): {len(medv_river)}")
print(f"   Sample size (No Charles River): {len(medv_no_river)}")
print(f"   Mean MEDV (Charles River): ${medv_river.mean():.2f}k")
print(f"   Mean MEDV (No Charles River): ${medv_no_river.mean():.2f}k")
print(f"   Difference in means: ${medv_river.mean() - medv_no_river.mean():.2f}k")
print(f"\n   T-statistic: {t_statistic:.4f}")
print(f"   P-value: {p_value:.4f}")

print("\n4. CONCLUSION:")
if p_value < 0.05:
    print(f"   Since p-value ({p_value:.4f}) < α (0.05), we REJECT the null hypothesis.")
    print(f"   \n   There IS a statistically significant difference in median home values.")
    print(f"   Houses bounded by the Charles River have significantly higher median values")
    print(f"   (${medv_river.mean():.2f}k) compared to those not bounded by the river (${medv_no_river.mean():.2f}k).")
    print(f"   \n   BUSINESS INSIGHT: Riverside location adds approximately ${medv_river.mean() - medv_no_river.mean():.2f}k")
    print(f"   to the median home value, representing a premium location factor.")
else:
    print(f"   Since p-value ({p_value:.4f}) >= α (0.05), we FAIL TO REJECT the null hypothesis.")
    print(f"   There is no statistically significant difference in median home values.")

print("\n" + "="*80)

### 3.2 ANOVA: Is there a difference in median values for each AGE group?

In [ ]:
print("="*80)
print("ANOVA TEST")
print("="*80)

print("\n1. HYPOTHESIS:")
print("   H0 (Null Hypothesis): There is no significant difference in median home values")
print("                         across different age groups.")
print("   H1 (Alternative Hypothesis): At least one age group has a significantly different")
print("                                median home value.")

print("\n2. SIGNIFICANCE LEVEL:")
print("   α = 0.05")

print("\n3. TEST STATISTICS:")

# Prepare data for ANOVA
group1 = boston_df[boston_df['AGE_Group'] == '35 years and younger']['MEDV']
group2 = boston_df[boston_df['AGE_Group'] == 'Between 35 and 70 years']['MEDV']
group3 = boston_df[boston_df['AGE_Group'] == '70 years and older']['MEDV']

# Perform ANOVA
f_statistic, p_value_anova = scipy.stats.f_oneway(group1, group2, group3)

print(f"   Group 1 (≤35 years): Mean = ${group1.mean():.2f}k, n = {len(group1)}")
print(f"   Group 2 (35-70 years): Mean = ${group2.mean():.2f}k, n = {len(group2)}")
print(f"   Group 3 (>70 years): Mean = ${group3.mean():.2f}k, n = {len(group3)}")
print(f"\n   F-statistic: {f_statistic:.4f}")
print(f"   P-value: {p_value_anova:.4f}")

# Alternative method using OLS
model = ols('MEDV ~ C(AGE_Group)', data=boston_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("\n   ANOVA Table:")
print(anova_table)

print("\n4. CONCLUSION:")
if p_value_anova < 0.05:
    print(f"   Since p-value ({p_value_anova:.4f}) < α (0.05), we REJECT the null hypothesis.")
    print(f"   \n   There IS a statistically significant difference in median home values")
    print(f"   across different age groups.")
    print(f"   \n   BUSINESS INSIGHT: The age of housing units significantly impacts home values.")
    print(f"   Management should consider the proportion of older housing stock when")
    print(f"   evaluating property values in different areas.")
else:
    print(f"   Since p-value ({p_value_anova:.4f}) >= α (0.05), we FAIL TO REJECT the null hypothesis.")
    print(f"   There is no statistically significant difference across age groups.")

print("\n" + "="*80)

### 3.3 Pearson Correlation: Relationship between NOX and INDUS

In [ ]:
print("="*80)
print("PEARSON CORRELATION TEST")
print("="*80)

print("\n1. HYPOTHESIS:")
print("   H0 (Null Hypothesis): There is no relationship between nitric oxide concentrations")
print("                         and proportion of non-retail business acres (ρ = 0).")
print("   H1 (Alternative Hypothesis): There IS a relationship between NOX and INDUS (ρ ≠ 0).")

print("\n2. SIGNIFICANCE LEVEL:")
print("   α = 0.05")

print("\n3. TEST STATISTICS:")
# Perform Pearson correlation test
correlation_coef, p_value_corr = scipy.stats.pearsonr(boston_df['NOX'], boston_df['INDUS'])

print(f"   Pearson Correlation Coefficient (r): {correlation_coef:.4f}")
print(f"   P-value: {p_value_corr:.6f}")
print(f"   Coefficient of Determination (r²): {correlation_coef**2:.4f}")
print(f"   (r² indicates that {correlation_coef**2*100:.2f}% of variance in NOX is explained by INDUS)")

# Interpretation of correlation strength
if abs(correlation_coef) < 0.3:
    strength = "weak"
elif abs(correlation_coef) < 0.7:
    strength = "moderate"
else:
    strength = "strong"

direction = "positive" if correlation_coef > 0 else "negative"

print(f"\n   Correlation Strength: {strength.upper()} {direction} correlation")

print("\n4. CONCLUSION:")
if p_value_corr < 0.05:
    print(f"   Since p-value ({p_value_corr:.6f}) < α (0.05), we REJECT the null hypothesis.")
    print(f"   \n   We CAN conclude that there IS a statistically significant relationship")
    print(f"   between nitric oxide concentrations and industrial areas.")
    print(f"   \n   The correlation is {strength} and {direction} (r = {correlation_coef:.4f}).")
    print(f"   \n   BUSINESS INSIGHT: Industrial development is strongly associated with")
    print(f"   higher pollution levels. This should be considered when evaluating")
    print(f"   residential property values and environmental quality in different areas.")
else:
    print(f"   Since p-value ({p_value_corr:.6f}) >= α (0.05), we FAIL TO REJECT the null hypothesis.")
    print(f"   We cannot conclude there is a relationship between NOX and INDUS.")

print("\n" + "="*80)

### 3.4 Regression Analysis: Impact of DIS on MEDV

In [ ]:
print("="*80)
print("REGRESSION ANALYSIS")
print("="*80)

print("\n1. HYPOTHESIS:")
print("   H0 (Null Hypothesis): Weighted distance to employment centres has no impact")
print("                         on median home values (β = 0).")
print("   H1 (Alternative Hypothesis): Weighted distance DOES impact median home values (β ≠ 0).")

print("\n2. SIGNIFICANCE LEVEL:")
print("   α = 0.05")

print("\n3. TEST STATISTICS:")

# Prepare data for regression
X = boston_df[['DIS']]  # Independent variable
y = boston_df['MEDV']   # Dependent variable

# Add constant to the model
X_with_const = sm.add_constant(X)

# Fit the regression model
regression_model = sm.OLS(y, X_with_const).fit()

# Print regression results
print(regression_model.summary())

# Extract key statistics
slope = regression_model.params['DIS']
intercept = regression_model.params['const']
r_squared = regression_model.rsquared
p_value_reg = regression_model.pvalues['DIS']

print("\n" + "="*80)
print("KEY REGRESSION RESULTS:")
print("="*80)
print(f"\n   Regression Equation: MEDV = {intercept:.4f} + {slope:.4f} * DIS")
print(f"\n   Intercept (β0): {intercept:.4f}")
print(f"   Slope (β1): {slope:.4f}")
print(f"   R-squared: {r_squared:.4f}")
print(f"   P-value for DIS: {p_value_reg:.6f}")

print("\n4. CONCLUSION:")
if p_value_reg < 0.05:
    print(f"   Since p-value ({p_value_reg:.6f}) < α (0.05), we REJECT the null hypothesis.")
    print(f"   \n   Weighted distance to employment centres DOES have a statistically significant")
    print(f"   impact on median home values.")
    print(f"   \n   INTERPRETATION:")
    print(f"   - For each additional unit increase in weighted distance to employment centres,")
    print(f"     the median home value {'increases' if slope > 0 else 'decreases'} by approximately ${abs(slope):.2f}k.")
    print(f"   - The model explains {r_squared*100:.2f}% of the variance in home values.")
    print(f"   \n   BUSINESS INSIGHT: {'Proximity' if slope < 0 else 'Distance'} to employment centres is a significant")
    print(f"   factor in home valuations. Properties {'closer' if slope < 0 else 'farther'} to employment hubs command")
    print(f"   {'higher' if slope < 0 else 'lower'} prices. This should be factored into property investment decisions.")
else:
    print(f"   Since p-value ({p_value_reg:.6f}) >= α (0.05), we FAIL TO REJECT the null hypothesis.")
    print(f"   Distance to employment centres does not have a statistically significant impact.")

print("\n" + "="*80)

#### Visualization of Regression Results

In [ ]:
# Create scatter plot with regression line
plt.figure(figsize=(10, 6))
plt.scatter(boston_df['DIS'], boston_df['MEDV'], alpha=0.5, edgecolors='black', linewidth=0.5)
plt.plot(boston_df['DIS'], regression_model.predict(X_with_const), color='red', linewidth=2, 
         label=f'Regression Line: y = {intercept:.2f} + {slope:.2f}x')
plt.xlabel('Weighted Distance to Employment Centres', fontsize=12)
plt.ylabel('Median Home Value ($1000s)', fontsize=12)
plt.title('Regression Analysis: Impact of Distance to Employment Centres on Home Values', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Executive Summary for Upper Management

### Key Findings:

1. **Charles River Premium**: Houses bounded by the Charles River have significantly higher values, representing a premium location factor.

2. **Age Impact**: The age composition of housing units significantly affects property values, with different patterns observed across age groups.

3. **Environmental Concerns**: Strong positive correlation between industrial areas and pollution levels, which can impact residential desirability.

4. **Employment Access**: Distance to employment centres significantly impacts home values, indicating that accessibility to work locations is a key value driver.

### Recommendations:
- Properties near the Charles River represent premium investment opportunities
- Consider age distribution when evaluating neighborhoods
- Balance industrial development with environmental quality
- Proximity to employment centres should be weighted heavily in property valuations